## Bias correction

The aim is to debias the Climex2 data. ERA5-land is used as a reference here. The ibicus package is used here for bias correcting the data. The debias functions from ibicus require observations, model historical period, model future period. 

Here we are going to use the midday temperature values with the following time periods:
- Observation: ERA5-land  (1991 - 2010)
- Historical: CLIMEX (1991- 2010)
- Future: CLIMEX 20 years to select to bias correct


### User input

The user input required is:
- Future time period (20 years is required, provide start date)
- Bias correcting method
- Ensemble for Climex2 data
- Variable to debias + corresponding path 

In [2]:
import sys
import glob
import xarray as xr
import numpy as np
from dask.distributed import LocalCluster
from ibicus.debias import LinearScaling, DeltaChange, QuantileMapping, CDFt, ISIMIP, ScaledDistributionMapping
import time
import dask.array as da 
import os
import matplotlib.pyplot as plt

In [3]:
t0 = time.time()
baseline_ens = 'fpp'
ens = 'fpp'
bias_correct = 'QM'
start_year = '1991'
end_year = '2010'
var = 'tas'


In [4]:
# Used for visualisation purposes
#client = LocalCluster().get_client()

C:\Users\ophme\AppData\Local\anaconda3\envs\wildfire\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55894 instead
  warnings.warn(
2025-03-28 16:43:12,227 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 7db96ca070678a52c637aa2a7fb30fbe initialized by task 'rechunk-merge-rechunk-split-rechunk-transfer-shuffle-barrier-7db96ca070678a52c637aa2a7fb30fbe' executed on worker tcp://127.0.0.1:55914
2025-03-28 16:43:12,429 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4bd75d545f3b62caa640213db7a8edf1 initialized by task ('rechunk-merge-rechunk-split-rechunk-transfer-f53700386c3eaae5fdd844989623e923', 0, 0, 1, 0, 0, 1) executed on worker tcp://127.0.0.1:55914
2025-03-28 16:43:12,490 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle e81bb3e1107a5b00b92943665a9914eb initialized by task ('rechunk-merge-rechunk-split-rechunk-transfer-f53700386c3eaae5fdd

In [5]:
#client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55894/status,
Dashboard: http://127.0.0.1:55894/status,Workers: 4
Total threads: 12,Total memory: 31.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55895,Workers: 4
Dashboard: http://127.0.0.1:55894/status,Total threads: 12
Started: Just now,Total memory: 31.65 GiB
Comm: tcp://127.0.0.1:55923,Total threads: 3
Dashboard: http://127.0.0.1:55924/status,Memory: 7.91 GiB
Nanny: tcp://127.0.0.1:55898,


### 2. Load files & preprocess data


#### Load climex data - historical (1991 - 2010)

In [6]:
years = np.arange(1991, 2011)
path = "O:/Public/sharing-4270-CERM/VLYMI/CLIMEX2/GlobusDownload/ClimExII_4_Friends/ClimExII_4_Friends/"

files = []

for year in years:
    year = str(year)
    files = files + sorted(glob.glob(path +'/' + baseline_ens + '/' + year + '/' + var + '*.nc'))[5:10] # Select only June to October

    

In [7]:
len(files)/5 # Only considering 5months / year

20.0

In [8]:
%%time
ds_hist = xr.open_mfdataset(files)


CPU times: total: 51.5 s
Wall time: 1min 48s


In [9]:
# Select midday timestep
ds_hist = ds_hist.tas[4:, :, :][::8] 

In [10]:
ds_hist = ds_hist.chunk(chunks={'rlat': 20, 'rlon': 20, 'time': -1})

In [11]:
len(ds_hist.time)

3060

#### Load "observations" - ERA5-land (1991 - 2010)

In [12]:
years = np.arange(1991, 2011)
path = 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/ERA5-land/regrided/FWI-variables/'

files = []

for year in years:
    year = str(year)
    if var == 'tas':
        var_era5 = 'temp'
    files = files + sorted(glob.glob(path + var_era5 + '-' + year +'*.nc'))[5:10] # Select only June to October


len(files)/5 # Only considering 5months / year

20.0

In [13]:
%%time
ds_obs = xr.open_mfdataset(files).temp + 273.15

CPU times: total: 13.9 s
Wall time: 52 s


In [14]:
# Remove leap days 
ds_obs = ds_obs.sel(time = ~((ds_obs.time.dt.month == 2) & (ds_obs.time.dt.day == 29)))
ds_obs = ds_obs.chunk(chunks={'y': 20, 'x': 20, 'time': -1})

In [15]:
len(ds_obs.time)

3060

### 3. Future data to bias correct: Select 20 year period

In [16]:
years = np.arange(int(start_year), int(end_year) + 1)
path = "O:/Public/sharing-4270-CERM/VLYMI/CLIMEX2/GlobusDownload/ClimExII_4_Friends/ClimExII_4_Friends/"

files = []

for year in years:
    year = str(year)
    files = files + sorted(glob.glob(path +'/' + ens + '/' + year + '/' + var + '_' + ens + '*.nc'))[5:10] # Select only June to October

In [17]:
len(files)/5

20.0

In [18]:
%%time
ds_fut = xr.open_mfdataset(files)


CPU times: total: 40.1 s
Wall time: 1min 1s


In [19]:
# Select midday timestep
ds_fut = ds_fut.tas[4:, :, :][::8] 

In [20]:
ds_fut = ds_fut.chunk(chunks={'rlat': 20, 'rlon': 20, 'time': -1})

In [21]:
len(ds_fut.time)

3060

### 4. Debias data

Ibicus has an option to use dask to apply the debiaser which makes it possible to bias correct the data in Europe for a 20 year period. 

In [22]:
if bias_correct == 'QM':
    debiaser = QuantileMapping.from_variable(variable = var)
elif bias_correct == 'LS':
    debiaser = LinearScaling.from_variable(variable = var)
elif bias_correct == 'SDM':
    debiaser = ScaledDistributionMapping.from_variable(variable = var)
elif bias_correct == 'ISIMIP':
    debiaser = ISIMIP.from_variable(variable = var)
else:
    print('Not available. Options available are QM, LS, DC or SDM')
    

In [23]:
%%time 
collection = da.map_blocks(debiaser.apply, ds_obs.data, ds_hist.data, ds_fut.data, dtype=ds_obs.dtype, progressbar = False, parallel = False, failsafe = True)
debiased_cm_future = collection.compute(num_workers=8)

CPU times: total: 1min 5s
Wall time: 3min 9s


### 5. Save data

In [24]:
# Create folder to save data to 
save_path = 'O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/CLIMEX2/debiased/' + var + '/'+ ens + '/'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [25]:
ds_fut_new = ds_fut.copy()
ds_fut_debias = ds_fut_new.to_dataset()
ds_fut_debias = ds_fut_debias.drop_vars(var)
ds_fut_debias = ds_fut_debias.assign(tas = (['time', 'rlat', 'rlon'], debiased_cm_future))
ds_fut_debias.to_netcdf(save_path + var + '_' + start_year + '-' + end_year + '_' + bias_correct +'.nc')

In [26]:
t1 = time.time()
print((t1 - t0)/60)

9.933610141277313


In [27]:
fut = ds_fut_debias.tas.values

In [28]:
hist = ds_hist.values


KeyboardInterrupt



In [ ]:
hist[np.isnan(fut)] = np.nan

In [ ]:
obs = ds_obs.values

In [ ]:
fig, ax = plt.subplots()
ax.pcolor(np.nanmean(hist, axis = 0) - np.nanmean(obs, axis = 0), vmin = -5, vmax = 5, cmap = 'bwr' )
ax.plot(200, 200, 'kx')
ax.plot(320, 160, 'ko')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex = True)
ax[0].hist(obs[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'obs', lw = 2, cumulative=True)
ax[0].hist(fut[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'QM', lw = 1.2, cumulative=True)
ax[0].hist(hist[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'climex', lw = 1.2, cumulative=True)
ax[0].legend()
ax[0].set_title('location x')

ax[1].hist(obs[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'obs', lw = 2, cumulative=True)
ax[1].hist(fut[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'QM', lw = 1.2, cumulative=True)
ax[1].hist(hist[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'climex', lw = 1.2, cumulative=True)
ax[1].legend()
ax[1].set_title('location o')
ax[1].set_xlim(270, 315)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex = True)
ax[0].hist(obs[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'obs', lw = 2)
ax[0].hist(fut[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'QM', lw = 1.2)
ax[0].hist(hist[:, 200, 200], bins = np.arange(270, 320, 1), histtype='step', label = 'climex', lw = 1.2)
ax[0].legend()
ax[0].set_title('location x')

ax[1].hist(obs[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'obs', lw = 2)
ax[1].hist(fut[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'QM', lw = 1.2)
ax[1].hist(hist[:, 160, 320], bins = np.arange(270, 320, 1), histtype='step', label = 'climex', lw = 1.2)
ax[1].legend()
ax[1].set_title('location o')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex = True)
ax[0].plot(obs[:, 200, 200], label = 'obs', lw = 2)
ax[0].plot(fut[:, 200, 200], label = 'QM', lw = 1.2)
ax[0].plot(hist[:, 200, 200], label = 'climex', lw = 1.2)
ax[0].legend()
ax[0].set_title('location x')

ax[1].plot(obs[:, 160, 320], label = 'obs', lw = 2)
ax[1].plot(fut[:, 160, 320], label = 'QM', lw = 1.2)
ax[1].plot(hist[:, 160, 320], label = 'climex', lw = 1.2)
ax[1].legend()
ax[1].set_title('location o')
ax[0].set_xlim(0, 154)

In [ ]:
day = 100
fig, ax = plt.subplots(2, 1, sharex = True)
a = ax[0].contourf(hist[day, :, :] - obs[day, :, :] , levels = np.arange(-15, 14, 2), cmap = 'bwr', extend = 'both' )
ax[0].plot(200, 200, 'kx')
ax[0].plot(320, 160, 'ko')
ax[1].contourf(fut[day, :, :] - obs[day, :, :] , levels = np.arange(-15, 14, 2), cmap = 'bwr', extend = 'both' )
ax[1].plot(200, 200, 'kx')
ax[1].plot(320, 160, 'ko')
fig.colorbar(a, ax = ax[:])
ax[0].set_title('historical - obs')
ax[1].set_title('QM - obs')


In [ ]:
print(np.nanmean(obs[:, 200, 200]))
print(np.nanmean(hist[:, 200, 200]))
print(np.nanmean(fut[:, 200, 200]))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.hist(debiased_cm_future.flatten(), bins = np.arange(240, 330, 2), histtype='step', label = 'climex QM')
ax.hist(hist.flatten(), bins = np.arange(240, 330, 2), histtype='step', label = 'climex')
ax.hist(obs.flatten(), bins = np.arange(240, 330, 2), histtype='step', label = 'obs')
ax.legend()

In [29]:
exit()